In [1]:
import openai
import instructor
import json
import os
import malaya
from pydantic import BaseModel, Field
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from unidecode import unidecode
import re
import random

minimum_len = 15

def simple_cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string).replace('\n', ' ').replace('--', ' ').replace('/', ' ')).strip()

/home/husein/3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/husein/3.9/lib/python3.9/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/3.9/lib/python3.9/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [2]:
openai.api_key = ''

In [3]:
# instructor.patch()

In [4]:
from typing import List

class Conversation(BaseModel):
    """
    generate conversation for user and assistant
    """
    user: str = Field(..., description = 'user question')
    assistant: str = Field(..., description = 'respond to user question')
        
class User(BaseModel):
    content: str = Field(..., description = 'user content')
        
class Assistant(BaseModel):
    content: str = Field(..., description = 'assistant content')

In [5]:
conversation_f = {
      "name": "conversation",
      "description": "generate conversation for user and assistant",
      "parameters": {
        "type": "object",
        "properties": {
          "user": {
            "type": "string",
            "description": "user question"
          },
          "assistant": {
            "type": "string",
            "description": "respond to user question"
          },
        },
        "required": ["user", "assistant"]
      }
    }

assistant_f = {
      "name": "assistant",
      "description": "respond to user question",
      "parameters": {
        "type": "object",
        "properties": {
          "respond": {
            "type": "string",
            "description": "respond to user question"
          },
        },
        "required": ["respond"]
      }
    }

In [6]:
def predict_user(messages):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k",
            response_model=User,
            messages=messages,
            temperature=1.0,
            max_tokens=1024,
            top_p=0.95,
        )
        return response.dict()
    except Exception as e:
        print(e)
        return None

def predict_assistant(messages):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k",
            response_model=Assistant,
            messages=messages,
            temperature=1.0,
            max_tokens=1024,
            top_p=0.95,
        )
        return response.dict()
    except Exception as e:
        print(e)
        return None
    
def predict(messages):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k",
            messages=messages,
            temperature=1.0,
            max_tokens=1024,
            top_p=1.0
        )
        return response.choices[0]['message']['content']
    except Exception as e:
        print(e)
        return None

In [7]:
system = 'Above is a conversation between a user and an intelligent assistant. You suppose to simulate conversation between the user and the assistant. Bear in mind your major request is to ask the assistant to generate some material. So you can ask the assistant either to make it more detailed, add more related information, or any other request to improve the generated material. Be creative and diverse in your request. Make the response short and the language casual.'
system_malay = 'Above is a conversation between a user and an intelligent assistant. You suppose to simulate conversation between the user and the assistant in malay language. Bear in mind your major request is to ask the assistant to generate some material. So you can ask the assistant either to make it more detailed, add more related information, or any other request to improve the generated material. Be creative and diverse in your request. Make the response short and the language casual.'
next_step = 'Above is a conversation between a user and an intelligent assistant. Now suppose you are the user, generate response according to the generated material to continue the conversation.'

In [8]:
# !rm -rf ultrachat-wikipedia
!mkdir ultrachat-wikipedia

mkdir: cannot create directory ‘ultrachat-wikipedia’: File exists


In [9]:
def template(rows):
    results = [rows[0]['content'] + '\n\n']
    for r in rows[1:]:
        results.append(f"{r['role']}: {r['content']}")
    return '\n'.join(results).strip()

In [10]:
def ultrachat(row, n = 1):
    results = [
        {'role': 'context', 'content': row['paragraph']},
        {'role': 'user', 'content': row['question']},
    ]
    initial = f"""
    {row['paragraph']}

    {row['question']}
    """.strip()
    messages = [
        {'role': 'system', 'content': system_malay},
        {'role': 'user', 'content': initial},
    ]
    r = predict(messages)
    results.append({
        'role': 'assistant', 'content': r,
    })
    messages.append({
        'role': 'assistant', 'content': r,
    })
    
    for _ in range(n):
    
        messages_temp = messages + [
            {'role': 'user', 'content': 'Now suppose you are the user, say something to continue the conversation based on given context. Make the response short and the language casual'}
        ]
        r2 = predict(messages_temp)
        if r2 is None:
            break

        results.append({
            'role': 'user', 'content': r2,
        })
        messages.append({
            'role': 'user', 'content': r2,
        })

        r = predict(messages)
        if r is None:
            break
        results.append({
            'role': 'assistant', 'content': r,
        })
        messages.append({
            'role': 'assistant', 'content': r,
        })
    
    return results

In [11]:
from glob import glob

files = sorted(glob('wikipedia-question/*.json'))

In [12]:
def generate(f):
    index = int(os.path.split(f)[1].replace('.json', ''))
    filename = f'ultrachat-wikipedia/{index}.json'
    if os.path.exists(filename):
        return
        
    with open(f) as fopen:
        data = json.load(fopen)
    
    if len(data['paragraph'].split()) < 100:
        return
    
    r = ultrachat(data, n = random.randint(1, 5))
    with open(filename, 'w') as fopen:
        json.dump(r, fopen)

In [13]:
generate(files[-1])

In [ ]:
from tqdm import tqdm

max_worker = 3
for i in tqdm(range(0, len(files), max_worker)):
    b = files[i: i + max_worker]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(generate, f): f for f in b}

        for future in as_completed(futures):
            future.result()

  0%|                                    | 12/31686 [09:07<521:39:09, 59.29s/it]